In [1]:
# import packages
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk  
import tensorflow as tf
!pip install transformers
import torch

# BERT and T5 are easily imported from Hugging Face's transformer library (https://huggingface.co/transformers/model_doc/bert.html#tfbertmodel
from transformers import BertTokenizer, TFBertModel, T5Tokenizer,  MT5Tokenizer, TFT5ForConditionalGeneration
from torch.utils.data import TensorDataset, random_split

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

# Keras functional API
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GlobalMaxPooling1D
from keras.layers import SpatialDropout1D
from keras.layers import Input

!pip install pyspellchecker  
from spellchecker import SpellChecker
  
import random
from random import randint
import more_itertools
from nltk import tokenize
nltk.download('punkt')

     |████████████████████████████████| 2.5MB 5.7MB/s 
     |████████████████████████████████| 901kB 29.6MB/s 
     |████████████████████████████████| 3.3MB 41.3MB/s 
     |████████████████████████████████| 2.7MB 5.0MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value=0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

from scipy.special import expit as sigmoid


In [3]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [4]:
# import transformers
#!pip install -q transformers

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [5]:
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test_set.tsv",sep='\t', encoding='ISO-8859-1')
dev = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/valid_set.tsv",sep='\t', encoding='ISO-8859-1')
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/training_set_rel3.tsv",sep='\t', encoding='ISO-8859-1')

In [ ]:
def tokenized(df, examples):
    total_ex = []
    window = 2
    for sentences in df['sent'][:examples]:
      if len(sentences) > window:
          window_list = list(more_itertools.windowed([k for k in range(len(sentences))],n=window, step=1))
      else:
          window_list = list(more_itertools.windowed([k for k in range(len(sentences))],n=len(sentences), step=1))
      ex = []
      for hi in window_list:
        encoded_dict = tokenizer(sentences[hi[0]], sentences[hi[1]], 
              max_length=75,
              add_special_tokens = True,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
        ex.append(encoded_dict)
      total_ex.append(ex)
    return total_ex

def coherence(df, examples):
  df['sent'] = df['essay'].apply(lambda x: tokenize.sent_tokenize(x))
  z = tokenized(df, examples)
  score = []
  for i in range(examples):
    print(i)
    bert_inputs = {'input_ids': tf.concat([k.input_ids for k in z[0]], axis=0),
                'token_type_ids': tf.concat([k.token_type_ids for k in z[0]], axis=0),
                'attention_mask': tf.concat([k.attention_mask for k in z[0]], axis=0)}

    bert_out = bert_model(bert_inputs)
    classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(bert_out[0])
    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(classification_token)
    classification = classification.numpy()
    cl_score = 0
    for cl in range(len(classification)):
      cl_score = cl_score + (-classification[cl]*np.log(classification[cl]) - (1-classification[cl])*np.log(1-classification[cl]))
    score.append(cl_score*(1/len(classification)))
  return score


In [ ]:
#a = np.random.shuffle(np.array(train['sent'][0]))
import random
coherence(train, 5)

0
1
2
3
4


[array([0.679084], dtype=float32),
 array([0.58351225], dtype=float32),
 array([0.5924472], dtype=float32),
 array([0.61584574], dtype=float32),
 array([0.6614047], dtype=float32)]

In [ ]:
tf.Tensor(
[[ 0.40502876  0.16268969 -0.18899728 ... -0.09224996  0.47692734
   0.36636263]
 [ 0.46364814  0.19616987 -0.27212214 ...  0.01225316  0.2651871
  -0.20405816]
 [ 0.1976564  -0.02260896 -0.2763127  ...  0.11219609  0.1907729
   0.11824741]
 ...
 [ 0.15325792  0.0064845  -0.43238774 ... -0.08453754  0.44622055
   0.3218247 ]
 [ 0.28758386 -0.07964344 -0.40736935 ...  0.0121686   0.60676616
   0.5432114 ]
 [ 0.70910037  0.41637385 -0.06621996 ... -0.39551225  0.84554166
   0.7115081 ]], shape=(15, 768), dtype=float32)
[array([0.35698232], dtype=float32)]


In [ ]:
bert_inputs = {'input_ids': tf.concat([k.input_ids for k in z[0]], axis=0),
                'token_type_ids': tf.concat([k.token_type_ids for k in z[0]], axis=0),
                'attention_mask': tf.concat([k.attention_mask for k in z[0]], axis=0)}

In [ ]:
bert_inputs['input_ids']

<tf.Tensor: shape=(15, 75), dtype=int32, numpy=
array([[  101, 12956,  1469, ...,     0,     0,     0],
       [  101, 11675,  1164, ...,     0,     0,     0],
       [  101,  1790,  1204, ...,     0,     0,     0],
       ...,
       [  101,  1192,  1547, ...,     0,     0,     0],
       [  101, 11056,  1190, ...,  2059,  1137,   102],
       [  101,  1986,   146, ...,     0,     0,     0]], dtype=int32)>

In [ ]:
sentences = train['sent'][0]
i = list(more_itertools.windowed([k for k in range(15)],n=2, step=1))
ex = []
for hi in i:
  encoded_dict = tokenizer(sentences[hi[0]], sentences[hi[1]], 
              max_length=75,
              add_special_tokens = True,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
  ex.append(encoded_dict)

In [ ]:
[i.input_ids for i in a]

[<tf.Tensor: shape=(1, 75), dtype=int32, numpy=
 array([[  101, 12956,  1469,  3054,   117,   146,  1341,  3154,  7565,
          1138,  1113,  1234,  1132,  1632,  3776,  4196,   120, 13974,
          1272,  1152,  1660,  1366,  1159,  1106, 13287,  1114,  2053,
           120,  1207,  1234,   117,  6618,  1366,  3858,  1164,  1103,
         12868,   113, 17338,   114,  1105,  7634,  1366,  1149,  1104,
           189,  2180,  2165,   106,   102, 11675,  1164,   106,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0]], dtype=int32)>,
 <tf.Tensor: shape=(1, 75), dtype=int32, numpy=
 array([[  101, 11675,  1164,   106,   102,  1790,  1204,  1128,  1341,
          1177,   136,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0, 

In [ ]:
w_sentences = ('hi im a good guy', 'whats up dude')

In [ ]:
tokenizer('hi im a good guy', 'whats up dude', truncation=True,max_length=50)

{'input_ids': [101, 20844, 13280, 170, 1363, 2564, 102, 1184, 1116, 1146, 17869, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
    
def normalize(df):
  df['normalized_score'] = df['domain1_score'] / df.groupby('essay_set')['domain1_score'].transform('max')
  return df
    
def tokenize_function(example):
  return tokenizer([x for x in example[:num_train_examples]], 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')

def fsm_stuff(hidden_size = 200, optimizer=tf.keras.optimizers.Adam()):
  
  """
  Build a simple classification model with BERT. Let's keep it simple and don't add dropout, layer norms, etc.
  """

  input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids_layer')
  token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='token_type_ids_layer')
  attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask_layer')

  bert_inputs = {'input_ids': input_ids,
                'token_type_ids': token_type_ids,
                'attention_mask': attention_mask}

  bert_out = bert_model(bert_inputs)


  classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(bert_out[0])

  
  hidden = tf.keras.layers.Dense(hidden_size, name='hidden_layer')(classification_token)

  classification = tf.keras.layers.Dense(8, activation='softmax',name='classification_layer')(hidden)

  classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], 
                                        outputs=[classification])
  
  classification_model.compile(optimizer=optimizer,
                          loss=tf.keras.losses.MeanAbsoluteError(name='mean_absolute_error'),
                          metrics=['accuracy'])


  return classification_model
    
def second_stage_model():
  return DecisionTreeClassifier(random_state=0)


# max_length = 512
# num_train_examples = 50
# """
#   Simplify dataframe and normalize scores based on essay set.
# """
# df = train
# df = df[['essay_set','essay','rater1_domain1','rater2_domain1','domain1_score']]
# df = normalize(df)


# """
#   Create features based on length, average/std of word length, unique words. 
# """

# df['length'] = df['essay'].str.split().str.len()
# df['avg_word_len'] = df['essay'].str.split().apply(lambda x: np.mean([len(y) for y in x]))
# df['std_word_len'] = df['essay'].str.split().apply(lambda x: np.std([len(y) for y in x]))
# df['unique_words'] = df['essay'].str.split().apply(lambda x: len(np.unique(x)))

# """
#   Run first stage of network with BERT and predict output.
# """
# x_train = tokenize_function(df['essay'].tolist())
# y_train = df.normalized_score[:num_train_examples]
# fsm = first_stage_model()
# fsm.fit([x_train.input_ids, x_train.token_type_ids, x_train.attention_mask], y_train, epochs=5, batch_size=8)
# # X_fsm = fsm.predict(x_train)

# # """
# #     Combine output of first-stage model with other features for second stage model.
# # """

# # X_ext = np.array([X_fsm, df['length'], df['avg_word_len'], df['std_word_len'], df['unique_words']])


# # """
# #     Run second stage model.
# # """

# # ssm = second_stage_model()
# # ssm.fit(X_ext, y_train) 

In [ ]:
x_train

{'input_ids': <tf.Tensor: shape=(50, 512), dtype=int32, numpy=
array([[  101, 12956,  1469, ...,     0,     0,     0],
       [  101, 12956,   137, ...,   117,  4029,   102],
       [  101, 12956,   117, ...,     0,     0,     0],
       ...,
       [  101,  4373,  1128, ...,  1129,  1240,   102],
       [  101, 12956,  1469, ...,     0,     0,     0],
       [  101,  6701,  1116, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(50, 512), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(50, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [ ]:
y_train

0     0.666667
1     0.750000
2     0.583333
3     0.833333
4     0.666667
5     0.666667
6     0.833333
7     0.833333
8     0.750000
9     0.750000
10    0.666667
11    0.666667
12    0.583333
13    0.500000
14    0.500000
15    1.000000
16    0.666667
17    0.666667
18    0.333333
19    0.500000
20    0.666667
21    0.250000
22    0.833333
23    0.916667
24    0.666667
25    0.750000
26    0.333333
27    0.750000
28    0.750000
29    0.666667
30    0.833333
31    0.833333
32    0.500000
33    0.666667
34    0.750000
35    0.833333
36    1.000000
37    0.666667
38    0.833333
39    0.583333
40    0.166667
41    0.666667
42    0.500000
43    0.666667
44    0.666667
45    0.666667
46    0.666667
47    0.916667
48    0.500000
49    0.416667
Name: normalized_score, dtype: float64

NEW MODEL FOR TRAINING

In [43]:
def get_adversarial(df, examples):
  adv_essays = []
  for example in range(examples):
    random_examples = [randint(0, len(df)-1) for p in range(0,10)]
    essay = []
    for random_example in random_examples:
      essay.append(random.choice(df['sent'].iloc[random_example]))
    adv_essays.append(essay)
  return pd.Series(adv_essays)


In [42]:
def clean_anonymization(essay):
  res=[]
  for i in essay.split():
    if i.startswith("@"):
      continue
    else:
      res.append(i)
  return ' '.join(res)

train['essay']=train['essay'].apply(lambda x:clean_anonymization(x))
# also remove from dev and test
dev['essay']=dev['essay'].apply(lambda x:clean_anonymization(x))
test['essay']=test['essay'].apply(lambda x:clean_anonymization(x))

In [30]:
def normalize(df):
    df['normalized_score'] = df['domain1_score'] / df.groupby('essay_set')['domain1_score'].transform('max')
    return df

def tokenized(df, examples):
    example_iter = [randint(0, len(df)-1) for p in range(0,examples)]
    input_ids = []
    token_type_ids = []
    attention_mask = []
    gold_scores = []
    window = 2

    for iteration in example_iter:
      sentences = df['sent'].iloc[iteration]
      if len(sentences) > window:
          window_list = list(more_itertools.windowed([k for k in range(len(sentences))],n=window, step=1))
      else:
          window_list = list(more_itertools.windowed([k for k in range(len(sentences))],n=len(sentences), step=1))
      for hi in window_list:
        if len(hi) >= 2:
          encoded_dict = tokenizer(sentences[hi[0]], sentences[hi[1]], 
                max_length=75,
                add_special_tokens = True,
                truncation=True,
                padding='max_length', 
                return_tensors='tf')
        else:
          encoded_dict = tokenizer(sentences[hi[0]], 
                max_length=75,
                add_special_tokens = True,
                truncation=True,
                padding='max_length', 
                return_tensors='tf')
        input_ids.append(encoded_dict.input_ids)
        token_type_ids.append(encoded_dict.token_type_ids)
        attention_mask.append(encoded_dict.attention_mask)
        gold_scores.append(df['normalized_score'][iteration])
    
    adv = examples // 10 
    adv_essays = get_adversarial(df, adv)
    for adv_essay in adv_essays:
      window_list = list(more_itertools.windowed([k for k in range(len(adv_essay))],n=window, step=1))

      for hi in window_list:
        if len(hi) >= 2:
          encoded_dict = tokenizer(adv_essay[hi[0]], adv_essay[hi[1]], 
                max_length=75,
                add_special_tokens = True,
                truncation=True,
                padding='max_length', 
                return_tensors='tf')

        input_ids.append(encoded_dict.input_ids)
        token_type_ids.append(encoded_dict.token_type_ids)
        attention_mask.append(encoded_dict.attention_mask)
        gold_scores.append(0)
        example_iter.append('ADV')
    return {'gold_scores': gold_scores, 'input_ids': tf.concat([z for z in input_ids], axis=0), 'token_type_ids': tf.concat([z for z in token_type_ids], axis=0), 'attention_mask': tf.concat([z for z in attention_mask], axis=0), 'example': example_iter}


In [46]:
train = normalize(train)
train['sent'] = train['essay'].apply(lambda x: tokenize.sent_tokenize(x))

enc_train = tokenized(train, 100)
fsm = first_stage_model(train_layers = 0)
fsm.fit([enc_train['input_ids'], enc_train['token_type_ids'], enc_train['attention_mask']], pd.Series(enc_train['gold_scores']), epochs=5, batch_size=100, shuffle=True)


Epoch 1/5
16/16 [==============================] - 405s 24s/step - loss: 0.1833 - accuracy: 0.0756
Epoch 2/5
16/16 [==============================] - 392s 24s/step - loss: 0.1833 - accuracy: 0.0756
Epoch 3/5
 6/16 [==========>...................] - ETA: 4:20 - loss: 0.1832 - accuracy: 0.0583

KeyboardInterrupt: ignored

In [ ]:
fsm.predict([enc_train['input_ids'][20:30], enc_train['token_type_ids'][20:30], enc_train['attention_mask'][20:30]])

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [45]:
def first_stage_model(hidden_size = 200, train_layers = -1, 
 optimizer=tf.keras.optimizers.RMSprop(0.000005)):
    
    """
    Build a simple classification model with BERT. Let's keep it simple and don't add dropout, layer norms, etc.
    """

    input_ids = tf.keras.layers.Input(shape=(75,), dtype=tf.int32, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(75,), dtype=tf.int32, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(75,), dtype=tf.int32, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                  'token_type_ids': token_type_ids,
                  'attention_mask': attention_mask}
    
    #restrict training to the train_layers outer transformer layers
    if not train_layers == -1:

            retrain_layers = []

            for retrain_layer_number in range(train_layers):

                layer_code = '_' + str(11 - retrain_layer_number)
                retrain_layers.append(layer_code)

            for w in bert_model.weights:
                if not any([x in w.name for x in retrain_layers]):
                    w._trainable = False


    bert_out = bert_model(bert_inputs)


    classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(bert_out[0])

    
    hidden = tf.keras.layers.Dense(hidden_size, name='hidden_layer')(classification_token)

    classification = tf.keras.layers.Dense(1, activation='softmax',name='classification_layer')(hidden)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], 
                                          outputs=[classification])
    
    classification_model.compile(optimizer=optimizer,
                            loss=tf.keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"),
                            metrics=['accuracy'])


    return classification_model